In [34]:
import torch
from torchvision import models

In [35]:
x: torch.Tensor = torch.randn(1, 3, 224, 224).cuda()

In [36]:
model: torch.nn.Module = models.shufflenet_v2_x0_5(weights=models.ShuffleNet_V2_X0_5_Weights.DEFAULT).cuda()

In [37]:
# half
half = True

In [38]:
# 动态batch
dynamic_batch = True

In [39]:
onnx_path = "./models/shufflenet_v2_x0_5"
if dynamic_batch:
    onnx_path += "-dynamic_batch"
if half:
    onnx_path += "-half"
    model: torch.nn.Module = model.half()
    x: torch.Tensor = x.half()
onnx_path += ".onnx"
onnx_path

'./models/shufflenet_v2_x0_5-dynamic_batch-half.onnx'

In [40]:
torch.onnx.export(
    model=model,
    args=x,
    opset_version=17,
    f=onnx_path,
    input_names=["images"],
    output_names=["classes"],
    dynamic_axes={"images": {0: "batch_size"}, "classes": {0: "batch_size"}} if dynamic_batch else None,
)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [41]:
import onnx
from onnxsim import simplify

In [42]:
model_ = onnx.load(onnx_path)

In [43]:
model__, ok = simplify(model_)
print(ok)

True


In [44]:
onnx.save(model__, onnx_path)